This file is used to develop a filter for bouys in regards to minimum distance to shore

This method uses shapelys buffer function and euqlidian distance functions and is only a rough approximation.

A better method would be to use geodesic or great-circle distance.

In [10]:
import cartopy
import shapely
from shapely.geometry import MultiPolygon, Polygon, Point
import geopy
import numpy as np
import os
import pickle
from pykml.factory import KML_ElementMaker as KML
import lxml

In [11]:
land_list = list(cartopy.feature.NaturalEarthFeature('physical', 'land', '50m').geometries())
polygon_list = []

for p  in land_list:
    if type(p) == MultiPolygon:
        polygon_list.extend(p.geoms)

    else:
        polygon_list.append(p)

land_multipolygon = MultiPolygon([p.buffer(0.01) for p in polygon_list])


In [12]:
#directory and filename where the pinmap filtering program saved its result
read_folder = './all_data_KML'
var_fn = 'result'

#Load datadir and dataframe
with open(os.path.join(read_folder, var_fn),'rb') as f_r:
    data_dir, df_files = pickle.load(f_r)

In [13]:
remove_index = df_files.apply(lambda x: land_multipolygon.contains(Point(x['LONG'],x['LAT'])), axis=1)

GEOSException: TopologyException: side location conflict at -73.865824551427664 45.57942689305461. This can occur if the input geometry is invalid.

In [ ]:
df_files_keep = df_files[~remove_index]
df_files_removed = df_files[remove_index]

In [ ]:
write_folder = './all_data_filtered_KML'
save_var_fn = 'result' #to save data_dir, df_files tuple
kml_pinmap_fn = 'kml_pinmap'

In [ ]:
#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

#Save datadir and resulting dataframe
with open(os.path.join(write_folder, save_var_fn),'wb') as f_w:
    pickle.dump((data_dir, df_files_keep, df_files_removed), f_w)

for df_files, name in [(df_files_keep, 'kept'),(df_files_removed, 'removed')]:
    #Create kml map
    KML_fldr = KML.Folder(
        KML.name('All bouys ' + name),
        KML.description("Filtered by distance to shore"),
    )

    #iterate over all unique variable name, file name pairs
    for index,row in df_files.iterrows():
        #extract longitude and latitude
        long = row['LONG']
        lat = row['LAT']
        
        #Create the pin
        pin = KML.Placemark(
            KML.name(row['LEN']),
            KML.description(row['file'] + '\n' + str(row['LEN']) + " datapoints"),
            KML.Point(
                KML.coordinates(str(long) + "," + str(lat))
            )
        )
        KML_fldr.append(pin)
        
    with open(os.path.join(write_folder, kml_pinmap_fn + '_' + name + '.kml'), 'w') as f_w:
        f_w.write(lxml.etree.tostring(KML_fldr, pretty_print=True).decode())